# This notebook is for generating R2RML File from CSV file
1. Use this file to upload your csv file to datababe snad generate R2RML file 
2. Use R2RML mapping tool to connect with your csv file and convert it to RDF
3. Upload your RDF file to Graph Database.

In [1]:
import os
import psycopg2
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout, Button, Box, interact, interactive, fixed, interact_manual, Checkbox
from PIL import Image
import pandas as pd
from collections import Counter

1. Read from local csv file
2. upload to local database (I used PostgresQL: https://www.postgresql.org/, PostgresQL in Python: http://initd.org/psycopg/ because R2RML mapping tool needs to extract data from database)

In [2]:
df=pd.read_csv("ProviderA_Data.csv", sep=',')
col = df.columns
print('The number of variables is ', len(col))

The number of variables is  8


In [3]:
variables = list(col)
varType = list(df.dtypes)

In [6]:
tableName_SQL = 'ProviderA_Data' # Give a table name
tableStructure = 'CREATE TABLE %s(' %tableName_SQL # SQL syntax to create a table 
for i in range(0, len(varType)):
    if varType[i] == 'int64':
        tempType = 'int'
    elif varType[i] == 'float64':
        tempType = 'real'
    else:
        tempType = 'VARCHAR'

    if str(variables[i]) == 'PRIMARY':
        temp = 'PRIMARYSCHOOL'+' '+tempType+','
    else:
        temp = str(variables[i])+' '+tempType+','
    tableStructure = tableStructure + temp 
tableStructure = tableStructure[:-1] + ')'

In [8]:
# Connect with PostgresQL
con = psycopg2.connect("host='localhost' dbname='changsun' user='postgres' password='postgres'") # host='CSdatabase', dbname='postgres'
print("Server Connected!\n")
cur = con.cursor()
# if it shows "relation "providera_data" already exists" please run next line of code
cur.execute("DROP TABLE ProviderA_Data")
cur.execute(tableStructure)
print("Darling, Table is created for you successfully.")

Server Connected!

Darling, Table is created for you successfully.


In [9]:
# Import CSV Data to Postgres
dataFile = "ProviderA_Data.csv" 
copy_sql = """
           COPY %s FROM stdin WITH (FORMAT CSV, HEADER true, NULL '', DELIMITER ',')           
           """ %tableName_SQL
with open(dataFile, 'r') as f:
    cur.copy_expert(sql=copy_sql, file=f)
con.commit()
cur.close()

In [10]:
style = {'description_width': 'initial'}
RDFTerm = ['child','class', 'column', 'constant', 'datatype', 'graph', 'graphMap', \
            'inverseExpression', 'joinCondition', 'language', 'parent', 'parentTriplesMap', \
            'sqlVersion', 'termtype', 'template']
tableClassTerm = ['sqlQuery', 'tableName']
def inputInfo():
    # set prefix
    uniLayout = Layout(width='70%', height='30px')
    
    common = "@prefix rr: <http://www.w3.org/ns/r2rml#>.\n@prefix ex: <http://example.com/base#>."
    prefixText = widgets.Textarea(value=common, description='Input Prefix', style=style,\
                                  layout=Layout(width='50%', height='100px'))

    # set map name
    common = '<#TriplesMap1>'
    mapName = widgets.Text(value=common, description='Give a name to this Map: ', style=style,\
                                  layout=uniLayout)

    # set name of logical table:
    tableClass = widgets.Dropdown(options=tableClassTerm, layout=uniLayout, \
                                   description='Choose Table Type: ', style=style)
    tableName = widgets.Textarea(value='SELECT * FROM %s;' %tableName_SQL, description='Table Name or SQL code:', \
                             layout=Layout(width='70%', height='70px'), style=style)
    
    # R2RML Properties
    subRDFTermSet = widgets.Dropdown(options=RDFTerm, layout=uniLayout, \
                                   description='Choose a RDF Term: ', style=style)
    
    # set subjectMap
    subWeb = widgets.Text(value='http://data.example.com/patient/', layout=uniLayout, \
                          description='Input your content:', style=style)
    subClass = widgets.Dropdown(options=list(col), \
                                description='Choose subject from logical table: ', \
                                layout=uniLayout, \
                                style=style)
    subName = widgets.Text(value='Patient', description='Give a name to this suject: ', \
                           layout=uniLayout, style=style)
    
    return prefixText, mapName, tableClass, tableName, subRDFTermSet, subWeb, subClass, subName

def inputObj(flagB):
    uniLayout = Layout(width='70%', height='30px')
    # set predicateObjectMap:
    objRDFTermSet = widgets.Dropdown(options=RDFTerm, layout=uniLayout, \
                                   description='Choose a RDF Term: ', style=style)
    objWeb = widgets.Text(value='PregnancyTimes', layout=uniLayout, \
                          description='Give a name to this predicate: ', style=style)
    objClass = widgets.Dropdown(options=list(col), layout=uniLayout, \
                                description='Choose object from logical table:', style=style)
    objNum = widgets.Text(value='1', layout=uniLayout, description='Number of Objects(Only integer): ', \
                          style=style)
    if flagB:
        return objRDFTermSet, objWeb, objClass, objNum
    else:
        return objRDFTermSet, objWeb, objClass     

In [12]:
prefixText, mapName, tableClass, tableName, subRDFTermSet, subWeb, subClass, subName = inputInfo()
objRDFTermSet, objWeb, objClass, objNum = inputObj(True)
para = [prefixText, mapName, tableName, subRDFTermSet, objRDFTermSet, subWeb, \
subClass, subName, objWeb, objClass]

group1 = [mapName, tableClass, tableName]
group2 = [subClass, subName, subRDFTermSet, subWeb]
group3 = [objWeb, objRDFTermSet, objClass, objNum]

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='70%')
box_layout2 = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='70%')
box1 = Box(children=group1, layout=box_layout)
box2 = Box(children=group2, layout=box_layout)
box3 = Box(children=group3, layout=box_layout)

steps = [prefixText, box1, box2, box3]
tab = widgets.Tab(children=steps, style=style)
tab.set_title(0, 'Step0--Prefix')
tab.set_title(1, 'Step1--Name')
tab.set_title(2, 'Step2--Subject')
tab.set_title(3, 'Step3--Pre&Obj')
    
action = widgets.Dropdown(options=['Add More Objects', 'Check Info', 'Generate File'],\
                          description='Action: ', style=style)
button = widgets.Button(description="RUN", button_style='danger')
boxACTION = Box(children=[action,button], layout=box_layout2)
display(tab, boxACTION)

add_objRDFTermSet=[None]*1000
add_objWeb=[None]*1000
add_objClass=[None]*1000
saveButton=[None]*1000

def actionFunction(x):
    
    if action.value == 'Add More Objects': 
        
        for i in range(0, int(objNum.value)-1):
            add_objRDFTermSet[i], add_objWeb[i], add_objClass[i] = inputObj(False) 
            saveButton[i] = widgets.Checkbox(value=False, \
                            description='I am sure to save this object', style=style)
            group4 = [add_objWeb[i], add_objRDFTermSet[i], add_objClass[i], saveButton[i]]
            add_box_layout = Layout(display='flex',
                            flex_flow='column',
                            border='solid',
                            align_items='stretch',
                            width='70%')
            box4 = Box(children=group4, layout=add_box_layout)
            display(box4)
        
    else:
        flag = True
        print('******** Check the information you inputed ********')
        for p in para:
            # check if all info were filled
            if not p.value:
                flag = False
                print('Please input all information!')

        if flag:
            add_preObjMap = ''
            tableView = '<#TableView> rr:sqlQuery """ %s """.'%tableName.value
            logTable = 'rr:logicalTable <#TableView>;' 
            subMap = """rr:subjectMap [\n\t\trr:%s "%s{%s}";\n\t\trr:class ex:%s;\n\t];"""\
            %(subRDFTermSet.value, subWeb.value, subClass.value, subName.value)
            preObjMap = """rr:predicateObjectMap [\n\t\trr:predicate ex:%s;\n\t\trr:objectMap [ rr:%s "%s" ];\n\t];"""%(objWeb.value, objRDFTermSet.value, objClass.value)
            for i in range(0, int(objNum.value)-1):
                if saveButton[i].value == True:
                    preObjMap = preObjMap+"""\n\trr:predicateObjectMap [\n\t\trr:predicate ex:%s;\n\t\trr:objectMap [ rr:%s "%s" ];\n\t];"""%(add_objWeb[i].value, add_objRDFTermSet[i].value, add_objClass[i].value)
                    
            fullText = prefixText.value+'\n\n'+tableView+'\n\n'+mapName.value+'\n\t'+logTable+'\n\t'+subMap\
                        +'\n\t'+preObjMap[:-1]+'.'
            if action.value=='Check Info':    
                print(fullText)
            if action.value=='Generate File':
                write_text = open("R2RML.ttl", "w")
                write_text.write(fullText)
                write_text.close()
                print('Your file: ', os.getcwd())
            
# interact_manual(actionFunction, x=action);
# doneButton.on_click(addObject)
button.on_click(actionFunction)

A Jupyter Widget

A Jupyter Widget